# Relational Model Initialization Scripts Creation

- First of all we need to import the necessary libraries.

In [1]:
import pandas as pd

- We will use initialization scripts to load the data into the database. The `postgis/postgis` image is based on the official `postgres` image with the addition of the PostGIS extensions. As per the documentation, initialization scripts will be executed in sorted name order. The `postgis/postgis` image already has an initialization script named `10_postgis.sh` that installs the PostGIS extensions. Also, we need an sql script that will create the tables to which we will insert the data, this file will be named `11_schema.sql`. For this reason we will name the sql scripts used for the data loading using the numbers `12-16`. Finally, we will also use an sql script named `17_indexes.sql` that will create the necessary indexes.

In [2]:
script_num = {
    'stores': 12,
    'products_1': 13,
    'products_2': 14,
    'products_3': 15,
    'products_4': 16
}

- We will read the store datasets.

In [3]:
stores_df_50k = pd.read_csv('./datasets/stores_50k.csv')
stores_df_100k = pd.read_csv('./datasets/stores_100k.csv')
stores_df = pd.read_csv('./datasets/stores_full.csv')

- We will create a script that will be used to insert the stores into the `Stores` table in the database.

In [4]:
def create_sql_insert_stores_script(df, dataset_size):
    quote = "'"
    escaped_quote = "''"
    with open(f'./sql_scripts/{dataset_size}/{script_num["stores"]}_stores.sql', 'w', encoding='utf-8') as stores_file:
        stores_file.write('INSERT INTO Stores (id, name, description, address, city, state, postal_code, location)\nVALUES\n')
        for idx, store in df.iterrows():
            stores_file.write(f'''\t({store["id"]}, '{store["name"]}', '{store["description"]}', '{store["address"].replace(quote, escaped_quote)}', '{store["city"].replace(quote, escaped_quote)}', '{store["state"]}', '{store["postal_code"]}', ST_MakePoint({store["longitude"]}, {store["latitude"]})::geography){',' if idx != (len(df) - 1) else ';'}\n''')

In [5]:
create_sql_insert_stores_script(stores_df_50k, dataset_size='50k')
create_sql_insert_stores_script(stores_df_100k, dataset_size='100k')
create_sql_insert_stores_script(stores_df, dataset_size='full')

- We will read the product datasets.

In [6]:
products_df_50k = pd.read_csv('./datasets/products_50k.csv')
products_df_100k = pd.read_csv('./datasets/products_100k.csv')
products_df = pd.read_csv('./datasets/products_full.csv')

- We will create 4 equal sized scripts that will be used to insert the products into the `Products` table in the database.

In [7]:
def create_sql_insert_products_scripts(df, dataset_size, num_files=4):
    chunk_size = len(df) // num_files
    for i in range(num_files):
        start = i * chunk_size
        end = (i + 1) * chunk_size if i < num_files - 1 else len(df)
        chunk = df.iloc[start:end]
        with open(f'./sql_scripts/{dataset_size}/{script_num[f"products_{i+1}"]}_products_{i+1}.sql', 'w', encoding='utf-8') as products_file:
            products_file.write('INSERT INTO Products (id, name, description, price, calories, carbs, fat, protein, store_id)\nVALUES\n')
            for idx, product in chunk.iterrows():
                products_file.write(f'''\t({product["id"]}, '{product["name"]}', '{product["description"]}', {product["price"]}, {product["calories"]}, {product["protein"]}, {product["carbs"]}, {product["fat"]}, {product["store_id"]}){',' if idx != (end - 1) else ';'}\n''')

In [8]:
create_sql_insert_products_scripts(products_df_50k, dataset_size='50k')
create_sql_insert_products_scripts(products_df_100k, dataset_size='100k')
create_sql_insert_products_scripts(products_df, dataset_size='full')